# CHISim Jupyter Notebook Entrance

This is the Jupyter Notebook Entrance for running experiments with CHISim.

## Step 1.Testbed Configuration

In [12]:
use_site "CHI@TACC"
export OS_PROJECT_NAME="CH-817567"

NODE_TYPE=compute_haswell
IMAGE=CC-Ubuntu20.04
EMAIL="$(openstack user show $USER -f value -c email)"

# A unique name for most provisioned resources to avoid collisions
RESOURCE_NAME="${USER}-jupyterhub-$(date +%b%d)"

[[ -n "$EMAIL" ]] || {
  echo >&2 "Could not look up your user, check your OS_PROJECT_NAME"
}

Now using CHI@TACC:
URL: https://chi.tacc.chameleoncloud.org
Location: Austin, Texas, USA
Support contact: help@chameleoncloud.org


## Step 2. Create a Lease

In [2]:
lease_name="$RESOURCE_NAME"
network_name="$RESOURCE_NAME"
public_network_id=$(openstack network show public -f value -c id)

blazar lease-create \
  --physical-reservation min=1,max=1,resource_properties="[\"=\", \"\$node_type\", \"$NODE_TYPE\"]" \
  --reservation resource_type=network,network_name="$network_name",resource_properties='["==","$physical_network","physnet1"]' \
  --reservation resource_type=virtual:floatingip,network_id="$public_network_id",amount=1 \
  --start-date "$(date +'%Y-%m-%d %H:%M')" \
  --end-date "$(date +'%Y-%m-%d %H:%M' -d'+1 day')" \
  "$lease_name"

# Wait for lease to start
timeout 30000 bash -c 'until [[ $(blazar lease-show $0 -f value -c status) == "ACTIVE" ]]; do sleep 1; done' "$lease_name" \
    && echo "Lease started successfully!"

#
# Fetch information about which resources were reserved for later use
#

reservations=$(blazar lease-show "$lease_name" -f json \
  | jq -r '.reservations')
host_reservation_id=$(jq -rs 'map(select(.resource_type=="physical:host"))[].id' <<<"$reservations")
fip_reservation_id=$(jq -rs 'map(select(.resource_type=="virtual:floatingip"))[].id' <<<"$reservations")

fip=$(openstack floating ip list --tags "reservation:$fip_reservation_id" -f json)
fip_id=$(jq -r 'map(.ID)[0]' <<<"$fip")
fip_addr=$(jq -r 'map(.["Floating IP Address"])[0]' <<<"$fip")

Matches: ('min=1,max=1', 'resource_properties', '["=", "$node_type", "compute_haswell"]')
Matches: ('min=1', 'max', '1')
Matches: (None, 'min', '1')
Matches: ('resource_type=network,network_name=kangz1-jupyterhub-Aug11', 'resource_properties', '["==","$physical_network","physnet1"]')
Matches: ('resource_type=network', 'network_name', 'kangz1-jupyterhub-Aug11')
Matches: (None, 'resource_type', 'network')
Matches: ('resource_type=virtual:floatingip,network_id=6189521e-06a0-4c43-b163-16cc11ce675b', 'amount', '1')
Matches: ('resource_type=virtual:floatingip', 'network_id', '6189521e-06a0-4c43-b163-16cc11ce675b')
Matches: (None, 'resource_type', 'virtual:floatingip')
Created a new lease:
+--------------+----------------------------------------------------------------------------+
| Field        | Value                                                                      |
+--------------+----------------------------------------------------------------------------+
| created_at   | 2021-08-1

## Step 3. Create Baremetal

In [3]:
# Ensure your Jupyter keypair is present
key_pair_upload

+-------------+------------------------------------------------------------------+
| Field       | Value                                                            |
+-------------+------------------------------------------------------------------+
| created_at  | 2021-08-11T20:44:30.000000                                       |
| deleted     | False                                                            |
| deleted_at  | None                                                             |
| fingerprint | 6d:83:e8:e7:f9:8c:6f:1c:2c:17:2c:96:db:99:d0:5a                  |
| id          | 9980                                                             |
| name        | kangz1-jupyter                                                   |
| updated_at  | None                                                             |
| user_id     | 249d942c9e24c3495c7c06b09825a998f231b0fab06d22d7babe230f1df0cce6 |
+-------------+------------------------------------------------------------------+


In [4]:
stack_name="$RESOURCE_NAME"
export OS_KEYPAIR_NAME="kangz1-jupyter"

openstack stack create "$stack_name" --wait \
  --template chameleon/jupyterhub_heat_template.yml \
  --parameter floating_ip="$fip_id" \
  --parameter reservation_id="$host_reservation_id" \
  --parameter key_name="$OS_KEYPAIR_NAME" \
  --parameter network_name="$network_name" \
  --parameter image="$IMAGE" && wait_ssh "$fip_addr"

2021-08-11 21:40:22Z [kangz1-jupyterhub-Aug11]: CREATE_IN_PROGRESS  Stack CREATE started
2021-08-11 21:40:22Z [kangz1-jupyterhub-Aug11.router]: CREATE_IN_PROGRESS  state changed
2021-08-11 21:40:23Z [kangz1-jupyterhub-Aug11.subnet]: CREATE_IN_PROGRESS  state changed
2021-08-11 21:40:24Z [kangz1-jupyterhub-Aug11.jupyterhub_ip]: ADOPT_IN_PROGRESS  state changed
2021-08-11 21:40:24Z [kangz1-jupyterhub-Aug11.jupyterhub_ip]: ADOPT_COMPLETE  state changed
2021-08-11 21:40:24Z [kangz1-jupyterhub-Aug11.jupyterhub_ip]: CHECK_COMPLETE  CHECK not supported for OS::Neutron::FloatingIP
2021-08-11 21:40:24Z [kangz1-jupyterhub-Aug11.subnet]: CREATE_COMPLETE  state changed
2021-08-11 21:40:24Z [kangz1-jupyterhub-Aug11.jupyterhub]: CREATE_IN_PROGRESS  state changed
2021-08-11 21:40:25Z [kangz1-jupyterhub-Aug11.router]: CREATE_COMPLETE  state changed
2021-08-11 21:40:25Z [kangz1-jupyterhub-Aug11.router_int]: CREATE_IN_PROGRESS  state changed
2021-08-11 21:40:28Z [kangz1-jupyterhub-Aug11.router_int]: CRE

: 124

## Step 4. Installing JupyterHub

The underlying base image does not have JupyterHub OS_KEYPAIR_NAME installed. To install and configure it, this example uses [Ansible](https://www.ansible.com/). First, some configuration of Ansible is required:

In [7]:
# Install Ansible dependencies
ansible-galaxy install -r ansible/requirements.yml

# Configure Ansible to run against provisioned nodes
sudo mkdir -p /etc/ansible
sudo tee /etc/ansible/hosts <<EOF
[jupyterhub]
$fip_addr ansible_user=cc ansible_become=yes ansible_become_user=root
EOF

Starting galaxy role install process
[WARNING]: - geerlingguy.docker (4.0.0) is already installed - use --force to
change version to unspecified
[WARNING]: - geerlingguy.pip (2.1.0) is already installed - use --force to
change version to unspecified
[jupyterhub]
 ansible_user=cc ansible_become=yes ansible_become_user=root


In [15]:
ansible-playbook --extra floating_ip="$fip_addr" --extra email_address="$EMAIL" ansible/bootstrap.yml
# ansible-playbook --extra floating_ip="129.114.109.241" --extra email_address="zhuangwei.kang@vanderbilt.edu" ansible/bootstrap.yml


PLAY [jupyterhub] **************************************************************

TASK [Gathering Facts] *********************************************************
[DEPRECATION WARNING]: Distribution ubuntu 20.04 on host 129.114.109.241 should
 use /usr/bin/python3, but is using /usr/bin/python for backward compatibility 
with prior Ansible releases. A future Ansible release will default to using the
 discovered platform python for this host. See https://docs.ansible.com/ansible
/2.10/reference_appendices/interpreter_discovery.html for more information. 
This feature will be removed in version 2.12. Deprecation warnings can be 
disabled by setting deprecation_warnings=False in ansible.cfg.
ok: [129.114.109.241]

TASK [geerlingguy.pip : Ensure Pip is installed.] ******************************
ok: [129.114.109.241]

TASK [geerlingguy.pip : Ensure pip_install_packages are installed.] ************
ok: [129.114.109.241] => (item=docker)
ok: [129.114.109.241] => (item=docker-compose)

TASK [

In [20]:
ansible-playbook ansible/configure.yml


PLAY [jupyterhub] **************************************************************

TASK [Gathering Facts] *********************************************************
[DEPRECATION WARNING]: Distribution ubuntu 20.04 on host 129.114.109.241 should
 use /usr/bin/python3, but is using /usr/bin/python for backward compatibility 
with prior Ansible releases. A future Ansible release will default to using the
 discovered platform python for this host. See https://docs.ansible.com/ansible
/2.10/reference_appendices/interpreter_discovery.html for more information. 
This feature will be removed in version 2.12. Deprecation warnings can be 
disabled by setting deprecation_warnings=False in ansible.cfg.
ok: [129.114.109.241]

TASK [Copy JupyterHub configuration.] ******************************************
changed: [129.114.109.241]

TASK [(read cryptography key)] *************************************************
changed: [129.114.109.241]

TASK [Check out NativeAuthenticator for JupyterHub.] *******

## Step 5. Install Depedencies for CHISim

In [ ]:
ansible-playbook ansible/chisim.yml